Wikipedia: https://en.wikipedia.org/wiki/Chebyshev_polynomials

Article: https://arxiv.org/pdf/2402.10132.pdf

The following is an implementation of a mathematical quantum expression, as appears on Page 8 of the attached article. It is based on the Chebyshev polynomials using the recurrence relation.

$$
B_L(t) = p_L(a,cos(t)) = a_0t+\frac{\sqrt{2}}\pi sin(\pi t)\sum_{k=0}^{L-1}\frac{a_k}k U_k(cos(\pi t))
$$

For $U_k$ being the recursive Chebyshev Polynomial.

In [ ]:
import scipy


def gaussian_discretization(num_qubits, mu=0, sigma=1, stds_around_mean_to_include=3):
    lower = mu - stds_around_mean_to_include * sigma
    upper = mu + stds_around_mean_to_include * sigma
    num_of_bins = 2**num_qubits
    sample_points = np.linspace(lower, upper, num_of_bins + 1)

    def single_gaussian(x: np.ndarray, _mu: float, _sigma: float) -> np.ndarray:
        cdf = scipy.stats.norm.cdf(x, loc=_mu, scale=_sigma)
        return cdf[1:] - cdf[0:-1]

    non_normalized_pmf = (single_gaussian(sample_points, mu, sigma),)
    real_probs = non_normalized_pmf / np.sum(non_normalized_pmf)
    return sample_points[:-1], real_probs[0].tolist()

In [ ]:
import numpy as np

from classiq import *

PI = np.pi
L = 4
TIME_STEPS = 1
NUM_QUBITS_GAUSSIAN = 1

MU = 1
SIGMA = 2

grid_points, probabilities = gaussian_discretization(NUM_QUBITS_GAUSSIAN)

In [ ]:
# Calculate sin(pi*x) and 2*cos(pi*x) using Taylor expansion


@qperm
def two_cos_pi_x(x: Const[QNum], out: Output[QNum]):
    out |= -2 * PI * (x - 0.5)  # expansion around x=0.5


@qperm
def sin_pi_x(x: Const[QNum], out: Output[QNum]):
    out |= -5 * (x - 0.5) ** 2 + 1  # expansion around x=0.5

# Chebyshev Polynomials

The Chebyshev polynomials are a sequence of orthogonal polynomials that are related to de Moivre's formula and the trigonometric functions. They are defined by the recurrence relation:

$$
U_0(x) = 1,
$$
$$
U_1(x) = 2x
$$
$$
U_{k+1}(x) = 2xU_k(x) - U_{k-1}(x)
$$

In [ ]:
@qperm
def uk(two_x: Const[QNum], uk_1: Const[QNum], uk_2: Const[QNum], uk: Output[QNum]):
    uk |= two_x * uk_1 - uk_2

# Brownian Motion

The Brownian motion is a stochastic process that models the random movement of particles in a fluid. The approximate solution in this context is the truncated Wiener series.

$$
B_L(t) = p_L(a,cos(t)) = a_0t+\frac{\sqrt{2}}\pi sin(\pi t)\sum_{k=0}^{L-1}\frac{a_k}k U_k(cost(\pi t))
$$

In [ ]:
@qperm
def truncated_wiener_series(
    a0: Const[QNum],
    a1: Const[QNum],
    a2: Const[QNum],
    a3: Const[QNum],
    t: Const[QNum],
    out: Output[QNum],
):
    two_cos_pi_t = QNum("two_cos_pi_t")
    sin_pi_t = QNum("sin_pi_t")
    U = [QNum(f"U{k}") for k in range(L)]
    two_cos_pi_x(x=t, out=two_cos_pi_t)
    U[0] |= 1
    U[1] |= two_cos_pi_t
    for k in range(2, L):
        uk(two_cos_pi_t, uk_1=U[k - 1], uk_2=U[k - 2], uk=U[k])

    sin_pi_x(x=t, out=sin_pi_t)

    out |= a0 * t + (2**0.5 / PI) * sin_pi_t * (a1 * U[1] + a2 * U[2] + a3 * U[3])

# Return to Price Space

The return to price space is a mathematical operation that converts the returns of a stock to its price. It is defined as:

$$
price = exp(\mu * returns + (\mu - \frac{\sigma^2}2)t)
$$

In [ ]:
@qperm
def return_to_price_space(
    returns: Const[QNum], t: Const[QNum], price: Output[QNum]
) -> None:
    price |= (
        SIGMA * returns + (MU - SIGMA**2 / 2) * t
    ) + 1  # TODO this is just tailor approximation for exp(SIGMA*x+(MU-SIGMA**2/2)*t) # TODO https://arxiv.org/pdf/2001.00807.pdf, inplace

In [ ]:
# TODO once https://classiq.atlassian.net/browse/CAD-19758 is done
# def max_G(G: QNum):
#   return 1/((2**((G.size-G.is_signed-G.fraction_digits)))-1)

# Putting it all together

In [ ]:
@qfunc
def main(t: Output[QNum], G: Output[QNum]):
    # Allocate qubits and prepare distributions
    B = QNum("B")
    As = [QNum(f"a{i}") for i in range(L)]
    for a in As:
        prepare_state(probabilities, 0, a)
    allocate(TIME_STEPS, t)
    hadamard_transform(t)

    # Create the truncated wiener series
    truncated_wiener_series(As[0], As[1], As[2], As[3], t, B)

    # Return to price space
    return_to_price_space(returns=B, t=t, price=G)

    for a in As[:4]:
        drop(a)

    # Amplitude Loading of the price
    # # TODO once https://classiq.atlassian.net/browse/CAD-19758 is done
    # ind_reg = QBit("ind_reg")
    # allocate(1, ind_reg)
    # ind_reg *= sqrt(1-G/max_G(G))

In [ ]:
qmod = create_model(
    entry_point=main,
    constraints=Constraints(optimization_parameter="width", max_width=139),
    preferences=Preferences(transpilation_option="none"),
)
qprog = synthesize(qmod)
show(qprog)

In [ ]:
write_qmod(qmod, "brownian_chebyshev_polynomials")